# Neural ODEs
Red Neuronal para simulacion de EDOs de primer ordel del tipo
$$\dfrac{d}{dt}S(t)=P_1(t) + P_2(t)*P_3(S)$$
donde $P_1$, $P_2$ y $P_3$ son polinomios.

In [29]:
# Recarga los modulos
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [30]:
import os

In [31]:
import warnings
warnings.filterwarnings('ignore')

In [32]:
import random

In [33]:
import gpustat
gpustat.print_gpustat()

Lucifer                                2022-11-26 22:11:21  526.98
[0] NVIDIA GeForce RTX 3080 Laptop GPU | 41°C,   0 % |   161 /  8192 MB | LUCIFER\Det-Pc(?M)


In [34]:
import numpy as mp
import pandas as pd

In [35]:
import matplotlib.pyplot as plt
%matplotlib inline

In [36]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils import data as data_torch

In [37]:
from tqdm import tnrange, tqdm_notebook

In [38]:
from api.utils import *

## Tratamiento de datos

In [39]:
# Obtenemos datos de las simulaciones
file_name = 'datos_ODEs_100.csv'
dir_file = 'C:/Users/Det-Pc/OneDrive/Documentos/GitHub/Proyecto_curso_DL/data'
data = get_ode_data(dir_file + '/' + file_name)
# data.head()

El tamano de los datos de polinomio en S y t es: 333274
El tamano de los datos de polinomio en S es    : 333811
El tamano de los datos de polinomio en t es    : 332915
El numero total de los datos es                : 1000000


In [40]:
# Extraemos los valores de las simulaciones
df_values = data['sol.y'].values

### DataSets

In [41]:
seq_len = 75
size_predic = len(data['sol.y'].values[0])
len_sim = df_values.shape[0]

In [42]:
# Creamos las sucesiones con los datos estandarizados
X_data, Y_data = get_seqs(data=df_values, seq_len=seq_len)

In [43]:
print(f'EL tamano de los datos las sucesiones_X es: {X_data.shape}')
print(f'EL tamano de los datos las sucesiones_Y es: {Y_data.shape}')

EL tamano de los datos las sucesiones_X es: (999925, 75)
EL tamano de los datos las sucesiones_Y es: (999925, 25)


In [44]:
# porcentaje de datos de entrenamiento
pct_data_train = 0.8

In [45]:
index_data = list(range(X_data.shape[0]))

train_index = random.sample(index_data, int(len(index_data)*pct_data_train))
val_test_index = list(set(index_data)-set(train_index))
val_index = random.sample(val_test_index, int(len(val_test_index)*0.5))
test_index = list(set(val_test_index)-set(val_index))

In [46]:
# Creamos las particiones
data_dic = {'train':X_data[train_index],\
           'val':X_data[val_index],\
           'test':X_data[test_index]}

In [47]:
class EDOs_dataloader(data_torch.Dataset):
    def __init__(self, edos_s):

        self.edos_s = edos_s
        
        
    def __getitem__(self, index):
        edo_i = self.edos_s[index]
        return edo_i
        

    def __len__(self):
        return len(self.edos_s)

In [48]:
partitions = ['train', 'val', 'test']
batch_size = 512
shuffle = True
lr = 0.001
edos_datasets = {x: EDOs_dataloader(data_dic[x]) for x in partitions}
dataloaders = {x: torch.utils.data.DataLoader(edos_datasets[x], batch_size=batch_size,\
                                              shuffle=shuffle) for x in partitions}
dataset_sizes = {x: len(edos_datasets[x]) for x in partitions}
print('Se cargo la data exitosamente!')
print(f'El tamano de la dataset es {dataset_sizes}')

Se cargo la data exitosamente!
El tamano de la dataset es {'train': 799940, 'val': 99992, 'test': 99993}


## Modelo

In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print('Estamos usando: {}'.format(device))

Estamos usando: cuda


In [99]:
class ODE_NET(nn.Module):
    def __init__(self, hidden_dim):
        super(ODE_NET, self).__init__()
        self.hidden_dim = hidden_dim
        self.conv1 = nn.Conv1d(in_channels=1,out_channels=32,kernel_size=6,stride=2,padding=3)
        self.relu = nn.ReLU()
        # self.pool = nn.MaxPool1d(2, 2)
        self.conv2 = nn.Conv1d(in_channels=32,out_channels=16,kernel_size=6,stride=2,padding=3)
        # self.fc1 = nn.Linear(16 * 5 * 5, 120)
        # self.fc2 = nn.Linear(120, 84)
        # self.fc3 = nn.Linear(84, 10)
        self.rnn1 = nn.LSTM(input_size=640, hidden_size=64, num_layers=1, batch_first=True)
        self.linear = nn.Linear(64, size_predic-seq_len)

    def forward(self, x, hidden):
        print(x.shape)
        x = F.relu(self.conv1(x))
        x = F.relu(self.conv2(x))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        print(x.shape)
        x = x.view(-1,)
        print(x.shape[0])
        x, hidden = self.rnn1(x, hidden)
        x = x.contiguous().view(-1, self.hidden_dim)
        # x = F.relu(self.fc1(x))
        # x = F.relu(self.fc2(x))
        x = self.linear(x)
        return x, hidden

    def init_hidden(self, batch_size):
        weight = next(self.parameters()).data
        hidden = (weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device),
                    weight.new(self.n_layers, batch_size, self.hidden_dim).zero_().to(device))
        return hidden


In [103]:
model = ODE_NET(hidden_dim=75)
model

ODE_NET(
  (conv1): Conv1d(1, 32, kernel_size=(6,), stride=(2,), padding=(3,))
  (relu): ReLU()
  (conv2): Conv1d(32, 16, kernel_size=(6,), stride=(2,), padding=(3,))
  (rnn1): LSTM(640, 64, batch_first=True)
  (linear): Linear(in_features=64, out_features=25, bias=True)
)

In [104]:
from torchsummary import summary
summary(model, (1,seq_len))

RuntimeError: Failed to run torchsummary. See above stack traces for more details. Executed layers up to: []